# Simple result overview on TC displacement risk

This notebook explores a few crude result outputs from the TC displacement risk computations

In [1]:
import pandas as pd
import glob
import os

path_results = '/cluster/work/climate/evelynm/IDMC_UNU/results/risk_tc/'
path_bem = '/cluster/work/climate/evelynm/IDMC_UNU/exposure/bem_cntry_files/'

In [2]:
all_iso3 = [cntry[0:3].upper() for cntry in os.listdir(path_bem)]

In [3]:
iso='TGO'

In [14]:
df_list=[]
for ssp in [245, 370, 585]:
    try:
        df_list.append(pd.read_csv(f'{path_results}{iso}/{iso}_ssp{ssp}_0.55_TC.csv', nrows=3))
    except:
        continue

In [22]:
cntry_df = pd.concat(df_list, axis=0).reset_index(drop=True)
cntry_df['iso3'] = iso
cntry_df.pop('Unnamed: 0')
cntry_df.pop('Exposure')
cntry_df

,Period,Scenario,AAD,RP_10,RP_25,RP_50,RP_100,RP_250,iso3
0,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0,TGO
1,2050,ssp245,0.0,0.0,0.0,0.0,0.0,0.0,TGO
2,2100,ssp245,0.0,0.0,0.0,0.0,0.0,0.0,TGO
3,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0,TGO
4,2050,ssp370,0.0,0.0,0.0,0.0,0.0,0.0,TGO
5,2100,ssp370,0.0,0.0,0.0,0.0,0.0,0.0,TGO
6,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0,TGO
7,2050,ssp585,0.0,0.0,0.0,0.0,0.0,0.0,TGO
8,2100,ssp585,0.0,0.0,0.0,0.0,0.0,0.0,TGO


In [31]:
global_list = []
for iso in all_iso3:
    cntry_list = []
    if os.path.exists(path_results+iso):
        for ssp in [245, 370, 585]:
            try:
                cntry_list.append(pd.read_csv(f'{path_results}{iso}/{iso}_ssp{ssp}_0.55_TC.csv', nrows=3))
                cntry_df = pd.concat(cntry_list, axis=0).reset_index(drop=True)
                cntry_df['iso3'] = iso
                cntry_df.pop('Unnamed: 0')
                cntry_df.pop('Exposure')
            except:
                continue
        global_list.append(cntry_df)

In [32]:
global_df = pd.concat(global_list, axis=0).reset_index(drop=True)
global_df

,Period,Scenario,AAD,RP_10,RP_25,RP_50,RP_100,RP_250,iso3
0,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0,RUS
1,2050,ssp245,0.0,0.0,0.0,0.0,0.0,0.0,RUS
2,2100,ssp245,0.0,0.0,0.0,0.0,0.0,0.0,RUS
3,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0,RUS
4,2050,ssp585,0.0,0.0,0.0,0.0,0.0,0.0,RUS
...,...,...,...,...,...,...,...,...,...
1366,2050,ssp245,0.0,0.0,0.0,0.0,0.0,0.0,RUS
1367,2100,ssp245,0.0,0.0,0.0,0.0,0.0,0.0,RUS
1368,hist,ERA-5,0.0,0.0,0.0,0.0,0.0,0.0,RUS
1369,2050,ssp585,0.0,0.0,0.0,0.0,0.0,0.0,RUS


In [33]:
global_df.to_csv(path_results+'TC_displ_admin0_med.csv')